In [ ]:
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from signals import *
from plots import *
from samplers import *
from pylab import rcParams
rcParams['figure.figsize'] = 5, 2.5

In [ ]:
# listdir("offline_results/")

In [ ]:
param = np.pi/18
generate_plots(5, 1, 0, 2*param)
plt.show()

In [ ]:
p = np.array([1.000000000000000000e+00,-2.825421682312351468e+00,-2.022133420929154990e-01,-1.483193386636804334e+00,-2.063111012904942587e+00,5.638963672080870015e-01,-3.154727063937948994e+00,1.396869864645395032e-01,-2.631363077020359764e+00,3.202233789725539292e+00])       

pol = SecondSurfacePolynomial(p)
t = np.linspace(-1,1, 100)
plt.plot(t, pol.get_samples(t), 'g', label="true")
sampler = SurfaceSampler(pol, 2*len(p), [param, 1, 1], interval_length=2, sigma=0.0, beg=-1)
plt.stem(pol.shifted_positions(sampler.sample_positions,[param, 1, 1]), sampler.sample_values, 'go', label="true samples")
plt.stem(sampler.sample_positions, sampler.sample_values, 'bo', label="samples given")
plt.stem(pol.shifted_positions(sampler.sample_positions,[0.05, 1, 1]), sampler.sample_values, 'ro', label="converged to")
plt.legend(loc=3)
plt.show()

In [ ]:
f1, f2 = generate_plots(7, 1, 1, np.degrees(2*param))
plt.title("evolution of error over time, degree 7, with noise")
f1.savefig("pol_iter_params.pdf")
f2.savefig("pol_iter_erros.pdf")
plt.show()

In [ ]:
generate_plots(7, 1, 0, 2*param)
plt.show()